In [190]:
#%%writefile data_df.py
import pandas as pd
import numpy as np

#make columns data def
def data_df(data, categ): 
    try:
        if categ in ('Bolig til salgs', 'Nye boliger', 'Fritidsbolig til salgs'):
            try:
                columns = ('name', 'from', 'address', 'square_metre', 'price', 'price_desc', 'r_type_amount', 'visning', 'link')
                df = pd.DataFrame(data, columns=columns)
            except: #if have not visning
                columns = ('name', 'from', 'address', 'square_metre', 'price', 'price_desc', 'r_type_amount', 'link')
                df = pd.DataFrame(data, columns=columns)
                
        elif categ in ('Fritidstomter', ):
            try:
                columns = ('name', 'from', 'address', 'square_metre', 'price', 'price_desc', 'visning', 'link')
                df = pd.DataFrame(data, columns=columns)
            except: #if have not visning
                columns = ('name', 'from', 'address', 'square_metre', 'price', 'price_desc', 'link')
                df = pd.DataFrame(data, columns=columns)

        elif categ in ('Tomter', 'Næringstomter'):
            try:
                columns = ('name', 'from', 'address', 'square_metre', 'price', 'price_desc', 'link')
                df = pd.DataFrame(data, columns=columns)
            except: #if have 'owner?'
                columns = ('name', 'from', 'address', 'square_metre', 'price', 'price_desc', 'owner?', 'link')
                df = pd.DataFrame(data, columns=columns)

        elif categ in ('Bolig til leie', 'Hjerterom - Bolig til leie', 'Næringseiendom til leie', ):
            columns = ('name', 'from', 'address', 'square_metre', 'price', 'r_type_amount', 'link')
            df = pd.DataFrame(data, columns=columns)

        elif categ in ('Næringseiendom til salgs', ):
            columns = ('name', 'from', 'address', 'square_metre', 'price', 'price_desc', 'r_type_amount', 'link')
            df = pd.DataFrame(data, columns=columns)
        elif categ in ('Bedrifter til salgs', ):
            columns = ('name', 'from', 'address', 'price', 'price_desc', 'link')
            df = pd.DataFrame(data, columns=columns)
        return df
    except:
        df = pd.DataFrame(data)
        return df

Overwriting data_df.py


In [1]:
from base import base
from collect import collect
#from connect_site import connect
#from cleaning import cleaning
from proxies import proxies
#from data_df import data_df
import pandas as pd

options = proxies()

In [161]:
f = base()

Сhoose a category:
1) Bolig til salgs
2) Tomter
3) Fritidsbolig til salgs
4) Fritidstomter
5) Bolig til leie
6) Hjerterom - Bolig til leie
7) Næringseiendom til salgs
8) Næringseiendom til leie
9) Næringstomter
10) Bedrifter til salgs
(pick number)
7
Do you want to select 1 - "cities" or 2 - "city with radius"? (pick number)
1
Choose the area\city(use comma and one space between if there is more than one)
Nordland
Accepted 1 cities out of 1


In [188]:
data = collect(*f, options=options) # stoped on 7

connection successful
All is collected!


In [163]:
all_data = data_df(data, f[0])

In [189]:
#%%writefile cleaning.py
import pandas as pd
import numpy as np

#cleaning data def
def cleaning(all_data): #from data_df
    
    # before cleaning
    amount = len(all_data)
    df = all_data.drop_duplicates()
    pd.set_option('display.max_colwidth', None)
    columns = ['name', 'from', 'address']
    print('Result:')
    
    #removing strange birds --------------------------------------------------------------------------------------
    
    if 'visning' in df.columns: #visning
        df['visning'].fillna('0', inplace=True)
        #is there any links
        html_v = df['visning'].str.contains('http')
        if html_v.sum():
            df.loc[html_v, 'link'] = df['visning'][html_v]
            df.loc[html_v, 'visning'] = '0'

    if 'r_type_amount' in df.columns: #r_type_amount
        df['r_type_amount'].fillna('0', inplace=True)
        ow = df['r_type_amount'].str.contains('http|finn') #links in r_type_amount
        df.loc[ow, 'link'] = df['r_type_amount'][ow]
        df.loc[ow, 'r_type_amount'] = '0'
        ow = df['r_type_amount'].str.contains('Visning') #'visning' in r_type_amount
        if 'visning' in df.columns:
            df.loc[ow, 'visning'] = df['r_type_amount'][ow]
        df.loc[ow, 'r_type_amount'] = '0'
        
    if 'price_desc' in df.columns: #price_desc
        df['price_desc'].fillna('0', inplace=True)
        ow = df['price_desc'].str.contains('http|finn') #links in 'price_desc'
        df.loc[ow, 'link'] = df['price_desc'][ow]
        df.loc[ow, 'price_desc'] = '0'
        ow = df['price_desc'].str.contains('Annet fritid|Eier|Leilighet|Hytte|soverom') #'r_type_amount' in 'price_desc'
        if 'r_type_amount' in df.columns:
            df.loc[ow, 'r_type_amount'] = df['price_desc'][ow]
        df.loc[ow, 'price_desc'] = '0'
        ow = df['price_desc'].str.contains('Visning') #'visning' in 'price_desc'
        if 'visning' in df.columns:
            df.loc[ow, 'visning'] = df['price_desc'][ow]
        df.loc[ow, 'price_desc'] = '0'
        
    if 'owner?' in df.columns: #owner? - tomter categ. not so many data(1-2 str), can't use 
        try:
            df['owner?'].fillna('0', inplace=True) #can't work with NaN
            ow = df['owner?'].str.contains('http|finn')
            df.loc[ow, 'link'] = df['owner?'][ow] #changing links columns
            del df['owner?']
        except Exception as e:
            print(f"Can't clean 'owner?' column- {e}")
    
    # if 'price' in df.columns: and if 'square_metre' in df.columns: ------------- nesessery
        

    #main cleaning -----------------------------------------------------------------------------------------------
    
    #for square_metre
    if 'square_metre' in df.columns:
        try:
            df['square_metre'] = df['square_metre'].str.split('-').str[-1] #clean from '-'
            df['square_metre'] = df['square_metre'].str[:-3] #clean from 'm2'
            df['square_metre'] = df['square_metre'].str.replace('\xa0', '') #clean from '\xa0'
            ow = df['square_metre'].str.isdigit()
            #if ~ow.sum(): #clean from not digit square_metre - usualy useless data
            #    df = df[ow]
            df['square_metre'] = df['square_metre'].map(int) #make int
            
        except Exception as e:
            print(f'problem with square_metre descr - {e}')
        
        columns.append('square_metre')
    
    #for price
    if 'price' in df.columns:
        try:
            df['price'] = df['price'].str.split('-').str[-1] #clean from '-'
            df['price'] = df['price'].str[:-3] #clean from 'kr'
            df['price'] = df['price'].str.replace('\xa0', '') #clean from '\xa0'
            ow = df['price'].str.isdigit()
            #if ~ow.sum(): #clean from not digit - probably allerede 'solgt'
            #    df = df[ow]
                
            #price to int
            try:
                df['price'] = df['price'].map(int)
                if len(df.loc[df['price'] < 100, 'price'])>0: #clean from untrust values
                    df.loc[df['price'] < 100, 'price'] =0
            except:
                print(f"can't make price int - {e}")

        except Exception as e:
            print(f'problem with price descr - {e}')
        
        columns.append('price')
        
    #for price_desc
    if 'price_desc' in df.columns:
        try:
            #trying to understend if is something to split
            df_pr = df['price_desc'].str.split(' ∙ ', expand=True) 
            df_pr_count = len(df_pr.columns)
            
            #if 2 items
            if df_pr_count == 2:
                df[['total_price', 'fellesutg']] = df_pr
                del df['price_desc']

                #fellestung cleaning
                try:
                    # clean from- :, -, kr, \xa0
                    df.loc[:, 'fellesutg'] = df['fellesutg'].str.split(':').str[-1].str.split('-').str[-1].str.replace('kr', '')
                    df.loc[:, 'fellesutg'] = df['fellesutg'].str.replace(r'^\s*$|^\xa0$', '0', regex=True).str.replace('\xa0', '')
                    try:
                        df['fellesutg'].fillna(0, inplace=True) #fill None with 0
                        df.loc[:, 'fellesutg'] = df['fellesutg'].map(int) #make int
                    except Exception as e:
                        print(f"can't make fellesutg int - {e}")
                        
                except Exception as e:
                    print(f'problem with fellesutg cleaning - {e}')
                
                columns.extend(['total_price', 'fellesutg'])
                
            #if nothing to split, probably it's a total
            if df_pr_count == 1:
                df['total_price'] = df_pr
                del df['price_desc']
                columns.append('total_price')
            #total_price cleaning
            try:
                # clean from- :, -, kr, \xa0
                df.loc[:, 'total_price'] = df['total_price'].str.split(':').str[-1].str.split('-').str[-1].str.replace('kr', '')
                df.loc[:, 'total_price'] = df['total_price'].str.replace(r'^\s*$|^\xa0$', '0', regex=True).str.replace('\xa0', '')
                try:
                    df['total_price'].fillna('0', inplace=True) #fill None with 0
                    df.loc[:, 'total_price'] = df['total_price'].map(int) #make int
                except Exception as e:
                    print(f"can't make total_price int - {e}")
                    
            except Exception as e:
                print(f'problem with total_price cleaning - {e}')
            
        except Exception as e:
            print(f'problem with price_desc descr - {e}')
            columns.append('price_desc')
            
    #for r_type_amount
    if 'r_type_amount' in df.columns:
        
        try:
            #trying to understend if is something to split
            df_am = df['r_type_amount'].str.split(' ∙ ', expand=True)
            df_am_count = len(df_am.columns)
            #if 2 items
            if df_am_count == 2:

                if df_am.iloc[:, -1].str.contains('Eier').sum(): #is there owner
                    df[['owner?', 'type']] = df_am
                    del df['r_type_amount']
                    columns.extend(['owner?', 'type'])
                    
                elif df_am.iloc[:, -1].str.split().str[0].str.isdigit().sum(): #is there rooms amount
                    df[['type', 'rooms_amount']] = df_am
                    del df['r_type_amount']
                    columns.extend(['type', 'rooms_amount'])
                    
            #if 3 items
            elif df_am_count == 3:
                df[['owner?', 'type', 'rooms_amount']] = df_am
                del df['r_type_amount']
                columns.extend(['owner?', 'type', 'rooms_amount'])

            #clean 'rooms_amount' from str -'rooms' and make int
            if 'rooms_amount' in df.columns:
                df.loc[:, 'rooms_amount'] = df['rooms_amount'].str.split().str[0] #clean from str 'rooms'

                try:
                    df['rooms_amount'].fillna('0', inplace=True) #fill None with 0
                    df.loc[:, 'rooms_amount'] = df['rooms_amount'].map(int) #make int
                except:
                    print("can't make rooms_amount int")
            
            if 'type' in df.columns:
                ow = df['type'].str.contains('Annet fritid')
                if ow.sum() and 'visning' in df.columns:
                    df.loc[ow, 'visning'] = df['type'][ow] 
                    df.loc[ow, 'type'] = '0'
            #more settings - sometimes nessesery
            df.loc[df['type'] == 'Garasje/Parker', ['type']] = 'Garasje/Parkering'
        
        except Exception as e:
            print(f'problem with r_type_amount descr - {e}')
            columns.append('r_type_amount')
    
    #for visning
    if 'visning' in df.columns:
        try:
            #clean from str - 'visning'
            df['visning'] = df['visning'].str.split(' - ').str[-1]
        except Exception as e:
            print(f'problem with visning descr - {e}')
        
        columns.append('visning')
    
    #sort columns ------------------------------------------------------------------------------------------------
    try:
        if df.columns[0] == 0:
            raise Exception("something wrong with columns names...")
        columns.append('link')
        df = df.reindex(columns, axis=1)
    except Exception as e:
        print(f'problem with columns sort - {e}')
    
    print(f'\n-- {amount - len(df)} -- str delited, if there is still a problem, try to clean in manually') # after cleaning
    return df

Overwriting cleaning.py


In [186]:
cleaning(all_data)

Result:
problem with price descr - cannot access local variable 'e' where it is not associated with a value
problem with r_type_amount descr - 'type'

-- 0 -- str delited, if there is still a problem, try to clean in manually


,name,from,address,square_metre,price,total_price,fellesutg,r_type_amount,link
0,Den gamle Husmorskolen i Alstahaug på Helgelandskysten med 2 skolebygninger og et stabbur er nå for salg.,Rede Eiendomsmegling AS,"Tjøttaveien 15 og 17, Sandnessjøen",1671,3200000,3200000,0,Eier (Selveier) ∙ Produksjon/Industri,https://www.finn.no/realestate/businesssale/ad.html?finnkode=266732508
1,"Hagstua, hytte- og leilighetsbygg, for salg.",NordMegler AS,"Hagskarveien 172, Leknes",1165,10000000,10251342,0,Eier (Selveier) ∙ Produksjon/Industri,https://www.finn.no/realestate/businesssale/ad.html?finnkode=311602568
2,Salg av AS - Attraktiv rorbuanlegg med meget idyllisk beliggenhet. BUD MOTTATT - KONTAKT MEGLER!,EiendomsMegler 1 Nord-Norge - Næring,"Besselvågveien 37, Sørvågen",6946,7000000,7001170,0,Eier (Selveier) ∙ Kombinasjonslokaler,https://www.finn.no/realestate/businesssale/ad.html?finnkode=311562870
3,Utviklingspotensial - Fullt utleid næringsbygg i sentrum av Stokmarknes - God profilering.,NordMegler AS,"Markedsgata 20B, Stokmarknes",1026,8500000,8500000,0,Eier (Selveier) ∙ Kontor,https://www.finn.no/realestate/businesssale/ad.html?finnkode=311267662
4,Tidligere Austbø barne - og ungdomsskole er nå til salgs | Helgelandskysten,Rede Eiendomsmegling AS,"Austbø Skole, Austbø",1552,500000,513864,0,Eier (Selveier) ∙ Undervisning/Arrangement,https://www.finn.no/realestate/businesssale/ad.html?finnkode=288287005
5,Unik næringseiendom ved foten av Reinefjorden - kommer for salg,NordMegler AS,"Moskenesveien 153, Reine",2507,Eier (Selveier) ∙ Produksjon/Indus,0,0,0,https://www.finn.no/realestate/businesssale/ad.html?finnkode=310559019
6,"Næringsseksjon Storgata 35 A, 1. etg. Legekontor.",NordMegler AS,"Storgata 35 A, Bodø",153,3890000,3988592,0,Eier (Selveier) ∙ Kontor,https://www.finn.no/realestate/businesssale/ad.html?finnkode=310538149
7,Ballstad havn - kombinasjonseiendom,Privat,"BALLSTADLANDET 204, Ballstad",2500,12000000,12000000,0,"Kombinasjonslokaler, Andre, Lager/Logistikk",https://www.finn.no/realestate/businesssale/ad.html?finnkode=309876175
8,Næringseiendommer (industri) beliggende i maritimt næringsområde på Steinesjøen i Bø kommune.,EiendomsMegler 1 Nord-Norge - Næring,"Steinesjøveien, Bø i Vesterålen",5694,7700000,7893670,0,Eier (Selveier) ∙ Produksjon/Industri,https://www.finn.no/realestate/businesssale/ad.html?finnkode=309992222
9,Ramberg - Forretningsbygg med flere utleieenheter,Proaktiv Eiendomsmegling Nord,"Flakstadveien 336, Ramberg",180,5000000,5126342,0,Eier (Selveier) ∙ Butikk/Handel,https://www.finn.no/realestate/businesssale/ad.html?finnkode=309751985


In [165]:
all_data

,name,from,address,square_metre,price,price_desc,r_type_amount,link
0,Den gamle Husmorskolen i Alstahaug på Helgelandskysten med 2 skolebygninger og et stabbur er nå for salg.,Rede Eiendomsmegling AS,"Tjøttaveien 15 og 17, Sandnessjøen",1 671 m²,3 200 000 kr,Totalpris: 3 200 000 kr,Eier (Selveier) ∙ Produksjon/Industri,https://www.finn.no/realestate/businesssale/ad.html?finnkode=266732508
1,"Hagstua, hytte- og leilighetsbygg, for salg.",NordMegler AS,"Hagskarveien 172, Leknes",1 165 m²,10 000 000 kr,Totalpris: 10 251 342 kr,Eier (Selveier) ∙ Produksjon/Industri,https://www.finn.no/realestate/businesssale/ad.html?finnkode=311602568
2,Salg av AS - Attraktiv rorbuanlegg med meget idyllisk beliggenhet. BUD MOTTATT - KONTAKT MEGLER!,EiendomsMegler 1 Nord-Norge - Næring,"Besselvågveien 37, Sørvågen",6 946 m²,7 000 000 kr,Totalpris: 7 001 170 kr,Eier (Selveier) ∙ Kombinasjonslokaler,https://www.finn.no/realestate/businesssale/ad.html?finnkode=311562870
3,Utviklingspotensial - Fullt utleid næringsbygg i sentrum av Stokmarknes - God profilering.,NordMegler AS,"Markedsgata 20B, Stokmarknes",1 026 m²,8 500 000 kr,Totalpris: 8 500 000 kr,Eier (Selveier) ∙ Kontor,https://www.finn.no/realestate/businesssale/ad.html?finnkode=311267662
4,Tidligere Austbø barne - og ungdomsskole er nå til salgs | Helgelandskysten,Rede Eiendomsmegling AS,"Austbø Skole, Austbø",1 552 m²,500 000 kr,Totalpris: 513 864 kr,Eier (Selveier) ∙ Undervisning/Arrangement,https://www.finn.no/realestate/businesssale/ad.html?finnkode=288287005
5,Unik næringseiendom ved foten av Reinefjorden - kommer for salg,NordMegler AS,"Moskenesveien 153, Reine",2 507 m²,Eier (Selveier) ∙ Produksjon/Industri,https://www.finn.no/realestate/businesssale/ad.html?finnkode=310559019,None,None
6,"Næringsseksjon Storgata 35 A, 1. etg. Legekontor.",NordMegler AS,"Storgata 35 A, Bodø",144 - 153 m²,3 890 000 kr,Totalpris: 3 988 592 kr,Eier (Selveier) ∙ Kontor,https://www.finn.no/realestate/businesssale/ad.html?finnkode=310538149
7,Ballstad havn - kombinasjonseiendom,Privat,"BALLSTADLANDET 204, Ballstad",2 500 m²,12 000 000 kr,Totalpris: 12 000 000 kr,"Kombinasjonslokaler, Andre, Lager/Logistikk",https://www.finn.no/realestate/businesssale/ad.html?finnkode=309876175
8,Næringseiendommer (industri) beliggende i maritimt næringsområde på Steinesjøen i Bø kommune.,EiendomsMegler 1 Nord-Norge - Næring,"Steinesjøveien, Bø i Vesterålen",5 694 m²,7 700 000 kr,Totalpris: 7 893 670 kr,Eier (Selveier) ∙ Produksjon/Industri,https://www.finn.no/realestate/businesssale/ad.html?finnkode=309992222
9,Ramberg - Forretningsbygg med flere utleieenheter,Proaktiv Eiendomsmegling Nord,"Flakstadveien 336, Ramberg",180 m²,5 000 000 kr,Totalpris: 5 126 342 kr,Eier (Selveier) ∙ Butikk/Handel,https://www.finn.no/realestate/businesssale/ad.html?finnkode=309751985


In [ ]:
#saving to csv
leie.to_csv('leie_120723.csv', sep=',', index=False, encoding='utf-8-sig')